In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2


--2020-10-23 10:56:05--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?AU2IPYWH8vuZuM9Mm3mxLx5X339hg4J2c2_D7JC885lUBxd-gFsBwb9PpDRWC1dvS66h-iMPf5RxhjeYVyBY6GTPjvb34AkF5qGg4ZzsLLMeyrwsqdRO2VnhyHcuyHdaUHd-frthiQRdfJ0FMmtuz-bZ8KoQFDCL666Id7den0GVjTsCsRX0rRX0CmLLI7A8KnWiPqeRo2iENJ4sJ54 [following]
--2020-10-23 10:56:05--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?AU2IPYWH8vuZuM9Mm3mxLx5X339hg4J2c2_D7JC885lUBxd-gFsBwb9PpDRWC1dvS66h-iMPf5RxhjeYVyBY6GTPjvb34AkF5qGg4ZzsLLMeyrwsqdRO2VnhyHcu

In [ ]:
!nvcc --version

/bin/bash: nvcc: command not found


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sa3avxdh
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sa3avxdh
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=15bb44d271efd931dba494bdcd11d363fcf381a746812415d4bfece619f25ba9
  Stored in directory: /tmp/pip-ephem-wheel-cache-421st2ez/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include<stdio.h>
#include <cstdlib> // malloc(), free()
#include <ctime>
#include <cmath>
#include <cuda.h>
#include <cuda_runtime_api.h>
#include<stdio.h>
#include<math.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
const int SIZE = 2000;
const int ITERS = 1;
void ErrorCheck(cudaError_t);
const int TILEWIDTH =32;
//making the non diagonal elements 0
__global__ void Subkernel(float* ipMat, unsigned int nRows, unsigned int nCols, float* opMat, int cRow)
 {
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  if (row < nRows && col < nCols) {
    if (row!=cRow) {
      opMat[row * nCols + col] = ipMat[row * nCols + col] - (ipMat[row*nCols + cRow] * ipMat[cRow*nCols + col]);
    }
  }
}

//making the diagonal elements 1
__global__ void ScaleKernel(float* ipMat, unsigned int nRows, unsigned int nCols, float* opMat, int cRow) 
{
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  
  if (row < nRows && col < nCols) {
     
    if (row == cRow && col == cRow) {
      
      opMat[cRow * nCols + cRow] = ipMat[cRow * nCols + cRow]/ipMat[cRow * nCols + cRow];
    }
    
    else if (row == cRow && col != cRow) {
      
      opMat[row * nCols + col] = ipMat[row * nCols + col] / ipMat[cRow*nCols + cRow];
    }
  }
}

bool GPU_ele(float** matrix, unsigned int nRows, unsigned int nCols, float** opMat, bool partialPivot) 
{
    clock_t start, end;
  float  timeGpu;
  cudaError_t status;
  float* Md;
  float* Rd;
 
  int bytes = nRows * nCols * sizeof(float);
  cudaMalloc((void**)&Md, bytes);
  cudaMalloc((void**)&Rd, bytes);
  status = cudaGetLastError();
  if (status != cudaSuccess)
  {
    ErrorCheck(status);
    cudaFree(Md);
    cudaFree(Rd);
    return false;
  }
  dim3 dimBlock(TILEWIDTH, TILEWIDTH);
  int grid1 = (int)ceil((float)nCols / (float)TILEWIDTH);
  int grid2 = (int)ceil((float)nRows / (float)TILEWIDTH);
  dim3 dimGrid(grid1,grid2 );
  for (int i = 0; i < nRows; i++)
  {
    cudaMemcpy(&Md[i*nCols], matrix[i], nCols*sizeof(float), cudaMemcpyHostToDevice);
  }
  cudaMemcpy(Rd, Md, bytes, cudaMemcpyDeviceToDevice);
 start = clock();
  for (int cRow = 0; cRow < nRows; cRow++) {
    ScaleKernel << <dimGrid, dimBlock >> > (Md, nRows, nCols, Rd,cRow);
    cudaThreadSynchronize();
    status = cudaGetLastError();
    if (status != cudaSuccess)
    {
      ErrorCheck(status);
      cudaFree(Md);
      cudaFree(Rd);
      return false;
    }
    cudaMemcpy(Md, Rd, bytes, cudaMemcpyDeviceToDevice);
    Subkernel << <dimGrid, dimBlock >> > (Md, nRows, nCols, Rd, cRow);
    cudaThreadSynchronize();
    status = cudaGetLastError();
    if (status != cudaSuccess)
    {
      ErrorCheck(status);
      cudaFree(Md);
      cudaFree(Rd);
      return false;
    }
    cudaMemcpy(Md, Rd, bytes, cudaMemcpyDeviceToDevice);
  }
end = clock(); 
  
  timeGpu = (float)(end - start)  / (float)CLOCKS_PER_SEC / ITERS;
 
  printf("\n(GPU)Time: %f sec\n", timeGpu); 
  for (int i = 0; i < nRows; i++)
  {
    cudaMemcpy(opMat[i], &Md[i * nCols], nCols * sizeof(float), cudaMemcpyDeviceToHost);
  }
  cudaFree(Md);
  cudaFree(Rd);
  //int h2=0;
 	printf("Final Values of the unknowns:\n");
  for(int h=0;h<nRows;h++)
  {
      for(int h2=nRows;h2<nCols;h2++)
      {
          printf("%f,",opMat[h][h2]);
      }
  }
  return true;
}

void ErrorCheck(cudaError_t status) { 
  printf("\nKernel failed!! %s", cudaGetErrorString(status));
}


int main()
{
  
  int rows,cols;
  rows = SIZE;
  cols = SIZE + 1;
  printf("Operating on a %d x %d matrix\n", SIZE, SIZE);
  float** a = new float* [rows];
  float** c_gpu = new float* [rows];
 
  for (int i = 0; i < rows; i++)
   {
    a[i] = new float[cols];
     c_gpu[i] = new float[cols];
  }
  for (int i = 0; i < rows; i++) 
  {
    for (int j = 0; j < cols; j++) 
    {
      a[i][j] = (float)rand() / (RAND_MAX);
    }
  }
/*
  bool success = GPU_ele(a, rows, cols, c_gpu, false);
  if (!success) { 
    printf("\nDevice Error!!");
    return 1; 
  }
 */
   for (int i = 0; i < ITERS; i++)
    { 
    GPU_ele(a, rows, cols, c_gpu, false);
  }  
  
}
